In [1]:
import protein_transformer
from protein_transformer.protein.PDB_Creator import PDB_Creator
from protein_transformer.dataset import VOCAB, prepare_dataloaders
import torch
from tqdm import tqdm

In [2]:
from protein_transformer.models.encoder_only import EncoderOnlyTransformer

In [3]:
chkpt = torch.load("../../../data/checkpoints/thirty2048c_best.chkpt")

In [4]:
chkpt.keys()

dict_keys(['model_state_dict', 'settings', 'epoch', 'optimizer_state_dict', 'scheduler_state_dict', 'loss', 'metrics', 'elapsed_time'])

In [5]:
model = EncoderOnlyTransformer(nlayers=6,
                               nhead=8,
                               dmodel=2048,
                               dff=2048,
                               max_seq_len=500,
                               dropout=0,
                               vocab=VOCAB,
                               angle_mean_path="../../../protein_transformer/protein/casp12_190927_100_angle_means.npy").cuda()

In [6]:
model.load_state_dict(chkpt["model_state_dict"])

<All keys matched successfully>

In [7]:
import os
os.chdir("/home/jok120/protein-transformer/protein_transformer/")

In [8]:
args = chkpt["settings"]

In [9]:
data = torch.load(args.data)
args.batch_size = 1
args.max_token_seq_len = data['settings']["max_len"]
training_data, validation_data, test_data = prepare_dataloaders(data, args, 500)
del data

In [10]:
from protein_transformer.log import init_metrics, angles_to_coords, inverse_trig_transform
from protein_transformer.losses import compute_batch_drmsd, \
    mse_over_angles, combine_drmsd_mse

In [11]:
def log_structure(args, pred_coords, gold_item, src_seq, name):
    """
    Logs a 3D structure prediction to wandb.
    # TODO save PDB files with numbers in addition to just GLTF files
    """
#     name = args.name
    gold_item_non_batch_pad = (gold_item != VOCAB.pad_id).any(dim=-1)
    gold_item = gold_item[gold_item_non_batch_pad]
    creator = PDB_Creator(pred_coords.detach().numpy(), seq=VOCAB.indices2aa_seq(src_seq.cpu().detach().numpy()))
    creator.save_pdb(f"../research/analysis/191125/{name}_pred.pdb", title="pred")
    gold_item[torch.isnan(gold_item)] = 0
    t_creator = PDB_Creator(gold_item.cpu().detach().numpy(), seq=VOCAB.indices2aa_seq(src_seq.cpu().detach().numpy()))
    t_creator.save_pdb(f"../research/analysis/191125/{name}_true.pdb", title="true")
    t_creator.save_gltfs(f"../research/analysis/191125/{name}_true.pdb", f"../research/analysis/191125/{name}_pred.pdb")
#     wandb.log({"structure_comparison": wandb.Object3D(f"../data/logs/structures/{args.name}_true_pred.gltf")}, commit=False)
#     wandb.log({"structure_prediction": wandb.Object3D(f"../data/logs/structures/{args.name}_pred.gltf")}, commit=True)

In [15]:
def eval_epoch(model, validation_data, device, args, metrics, mode="valid", pool=None):
    """
    One compete evaluation epoch.
    """
    model.eval()
    batch_iter = tqdm(validation_data, mininterval=.5, leave=False, unit="batch", dynamic_ncols=True) \
        if not args.cluster else validation_data

    if args.loss == "mse" and mode == "train":
        d_loss, ln_d_loss, r_loss = torch.tensor(0.), torch.tensor(0.), torch.tensor(0.)

    with torch.no_grad():
        for i, batch in enumerate(batch_iter):
            src_seq, tgt_ang, tgt_crds = map(lambda x: x.to(device), batch)
            pred = model(src_seq, tgt_ang)

#             if not (args.loss == "mse" and mode == "train"):
            d_loss, ln_d_loss, r_loss = compute_batch_drmsd(pred, tgt_crds, src_seq, return_rmsd=True,
                                                            do_backward=False, pool=pool)
            
            m_loss = mse_over_angles(pred, tgt_ang)
            c_loss = combine_drmsd_mse(ln_d_loss, m_loss, w=args.combined_drmsd_weight)
            print(i, m_loss.item(), d_loss.item())
            pred_coords = angles_to_coords(inverse_trig_transform(pred)[-1].cpu(), src_seq[-1].cpu(),
                remove_batch_padding=True)
            if d_loss.item() < 10:
                log_structure(args, pred_coords, tgt_crds, src_seq[-1], name=f"{mode}-{i}")


    return metrics

In [13]:
m = eval_epoch(model, validation_data, torch.device("cuda"), args, init_metrics(args), mode="valid")

  0%|          | 0/26 [00:00<?, ?batch/s]

0 0.19039252400398254 83.8077163696289
 PyMOL not running, entering library mode (experimental)


  4%|▍         | 1/26 [00:01<00:46,  1.84s/batch]

1 0.20813214778900146 54.3950309753418


  8%|▊         | 2/26 [00:03<00:40,  1.68s/batch]

...

16 0.22793811559677124 49.19363784790039
17 0.1852951943874359 36.691673278808594
18 0.1778576523065567 27.675121307373047


 77%|███████▋  | 20/26 [00:14<00:02,  2.42batch/s]

19 0.21293948590755463 30.152353286743164
20 0.08680253475904465 6.149044990539551
21 0.18325522541999817 15.835594177246094
22 0.15009233355522156 9.697128295898438


 92%|█████████▏| 24/26 [00:14<00:00,  3.02batch/s]

23 0.15357942879199982 11.212568283081055
24 0.15766416490077972 8.60629653930664
25 0.25827524065971375 11.07052230834961


In [16]:
m = eval_epoch(model, test_data, torch.device("cuda"), args, init_metrics(args), mode="test")

  0%|          | 0/40 [00:00<?, ?batch/s]

0 0.166086807847023 87.06140899658203


  2%|▎         | 1/40 [00:01<00:45,  1.17s/batch]

1 0.26264268159866333 53.96146011352539


  5%|▌         | 2/40 [00:02<00:42,  1.12s/batch]

...

 35%|███▌      | 14/40 [00:12<00:18,  1.37batch/s]

14 0.1710711419582367 40.88257598876953


 38%|███▊      | 15/40 [00:12<00:17,  1.44batch/s]

15 0.17690281569957733 70.60488891601562


KeyboardInterrupt: 

In [17]:
m = eval_epoch(model, training_data, torch.device("cuda"), args, init_metrics(args), mode="train")


  0%|          | 0/19804 [00:00<?, ?batch/s]

0 nan 41.85984802246094



  0%|          | 1/19804 [00:01<6:54:38,  1.26s/batch]

1 0.023436104878783226 59.780174255371094



  0%|          | 2/19804 [00:02<6:31:42,  1.19s/batch]

...

13405 0.0034312631469219923 7.309534549713135
13406 0.005337005015462637 9.92108154296875
13407 0.0023184074088931084 14.359464645385742



 68%|██████▊   | 13409/19804 [1:57:40<25:45,  4.14batch/s]

13408 0.0031548780389130116 11.689002990722656
13409 0.0038061589002609253 10.912062644958496
13410 0.0021437988616526127 8.552777290344238


AssertionError: The sequence length must match the coordinate length and contain 1 letter AA codes.1377 106


 68%|██████▊   | 13409/19804 [1:57:53<25:45,  4.14batch/s]

In [ ]:
import pymol

In [ ]:
pymol.cmd.load("../research/analysis/191125/test-38_true.pdb", "true")
pymol.cmd.load("../research/analysis/191125/test-38_pred.pdb", "pred")

In [ ]:
pymol.cmd.color("marine", "true")
pymol.cmd.color("oxygen", "pred")
pymol.cmd.align("true", "pred", quiet=True)

In [ ]:
pymol.cmd.save("../research/analysis/191125/t01.pse")